# Ad break detection and contextual Ad targeting

Contextual advertising is a form of targeted advertising where the advertisement is matched to the context of the webpage or media being consumed by the user. This process involves three key players: the publisher (website or content owner), the advertiser, and the consumer. Publishers provide the platform and content, while advertisers create ads tailored to the context. Consumers engage with the content, and relevant ads are displayed based on the context, creating a more personalized and relevant advertising experience.

![Ad decisions](./static/images/02-ad-decisions.png)

A challenging area of contextual advertising is inserting ads in media content for streaming on video on demand (VOD) platforms. This process traditionally relied on manual tagging, where human experts analyzed the content, identified breaks in the narrative and assigned relevant keywords or categories. However, this approach is time-consuming, subjective, and may not capture the full context or nuances of the content. Traditional AI/ML solutions can automate this process, but they often require extensive training data and can be expensive and limited in their capabilities.

Generative AI, powered by large language models, offers a promising solution to this challenge. By leveraging the vast knowledge and contextual understanding of these models, publishers can automatically generate contextual insights and taxonomies for their media assets. This approach streamlines the process and provides accurate and comprehensive contextual understanding, enabling effective ad targeting and monetization of media archives.

When you are done with this part of the workshop, you'll have created the following metadata for a video:
* a list of high quality ad placement opportunities  or _breaks_ available in the video
* contextual information for the video before and after each break, including classification using the IAB Content Taxonomy that is used by advertisers to classify content for automated placement using Ad Decision Servers.

# Prerequisites

To run this notebook, you need to have run the previous notebook:[01-video-time-segmentation](01-video-time-segmentation.ipynb), where you segmented the video using audio, visual and semantic information.

### Import python packages

In [ ]:
from pathlib import Path
import os
import json
import json
import boto3
import json_repair
import copy
from termcolor import colored
from IPython.display import JSON
from IPython.display import Video
from IPython.display import Pretty
from IPython.display import Image as DisplayImage
from lib.frames import VideoFrames
from lib.shots import Shots
from lib.scenes import Scenes
from lib.transcript import Transcript
#from lib.chapters import Chapters
from lib import bedrock_helper as brh
from lib import frame_utils
from lib import util
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO

### Retrieve saved values from previous notebooks
To run this notebook, you need to have run the previous notebook: 00_prerequisites.ipynb, where you installed package dependencies and gathered some information from the SageMaker environment.




In [ ]:
store -r

# Architecture

This hands-on workflow uses AWS services from SageMaker.  It takes scenes, conversation topics and advertising content taxonomies as inputs and produces contextual ad breaks and chapter segments as outputs.


![Contextual Ads workflow with outputs](./static/images/02-contextual-ads-workflow-w-outputs-drawio.png)


# Find ad placement opportunities by aligning scenes and topics to identify chapters in the narrative

In the [Video segmentation notebook](video-understanding-with-generative-ai-on-aws-main/01-video-time-segmentation.ipynb), we have separately processed the visual and audio cues from the video. Now, we will do one more step to bring them together and ensure that the transcription topics align with the scenes. The last thing you want is to insert an ad during an ongoing conversation or scene. To create alignment, we will iterate over each conversational topic, represented by its start and end timestamps, and a text description summarizing the topic. For each topic, the code identifies the relevant video scenes that overlap or fall within the topic's timestamp range. The output of this process is a list of chapters, where each chapter contains a list of scene IDs representing the video scenes that align with the corresponding audio conversation. After the alignment process, we have combined visual and audio cues into the final chapters. The breaks between chapters are ideal places for ad insertion because they occur between contextual changes in the content of the video. 

In real-world applications, we recommend surfacing these breaks as suggestions to the operator and having a human-in-the-loop step to confirm the final ad placements.

In [ ]:
import copy
from lib import frame_utils
import os


class Chapters:
    def __init__(self, topics, scenes, frames):
        self.video_asset_dir = frames.video_asset_dir()
        self.chapters = self.align_scenes_in_chapters(topics, scenes, frames)
        
    def align_scenes_in_chapters(self, topics, scenes, frames):
        scenes = copy.deepcopy(scenes)
    
        chapters = []
        for topic in topics:
            
            topic_start_ms = topic['start_ms']
            topic_end_ms = topic['end_ms']
            text = topic['reason']

            print(f"Topic: time { topic_start_ms } to { topic_end_ms  }, text { text }")

            # find all the frames that align with the conversation topic
            stack = []
            while len(scenes) > 0:
                scene = scenes[0]
                frame_start = scene['start_ms']
                frame_end = scene['end_ms']

                
                if frame_start > topic_end_ms:
                    print(f"break")
                    # topic overlaps scenes that belong to previous topic - merge the text
                    if not stack:
                        num_chapters = len(chapters)
                        if num_chapters > 0:
                            chapters[num_chapters-1]['text'] = chapters[num_chapters-1]['text'] + ' ' + text
                        
                    break
    
                # scenes before any conversation starts
                if frame_end < topic_start_ms:
                    print(f"scenes before any conversation starts")
                    chapter = Chapter(len(chapters), [scene], frames).__dict__
                    chapters.append(chapter)
                    scenes.pop(0)
                    continue
    
                stack.append(scene)
                scenes.pop(0)
    
            if stack:
                print(f"if stack")
                chapter = Chapter(len(chapters), stack, frames, text).__dict__
                chapters.append(chapter)
    
        ## There could be more scenes without converations, append them
        for scene in scenes:
            chapter = Chapter(len(chapters), [scene], frames).__dict__
            chapters.append(chapter)
    
        return chapters

class Chapter:
    def __init__(self, chapter_id, scenes, frames, text = ''):
        self.scene_ids = [scene['id'] for scene in scenes]
        self.start_frame_id = scenes[0]['start_frame_id']
        self.end_frame_id = scenes[-1]['end_frame_id']
        self.start_ms = scenes[0]['start_ms']
        self.end_ms = scenes[-1]['end_ms']
        self.id = chapter_id
        self.text = text
        #folder = os.path.join(frames.video_asset_dir(), 'chapters')
        #os.makedirs(folder, exist_ok=True) 
        self.composite_images = frames.create_composite_images(frames.frames[self.start_frame_id:self.end_frame_id+1], 'chapters', prefix="chapter_")
        
        return 


In [ ]:
video['chapters'] = Chapters(video['topics'], video['scenes'].scenes, video['frames'])

Examine the results

In [ ]:
display(JSON(video['chapters'].chapters))

#### Visualize the chapters

Now let's visualize the frames and the text for each of the chapters.  These will be the input to our prompt to generate contextual information for the ad breaks. Note that some chapters will not have any text associated with them.

<div class="alert alert-block alert-info">
💡 Use the scroll bar in the output box to view the chapters.  Some chapters contain more frames than can fit on a single composite image, so the may be multiple composite images displayed for each chapter.
</div>


In [ ]:
# visualize the chapters

STOP=10
for counter, b in enumerate(video["chapters"].chapters):
    print(f'\nChapter {counter}: frames {b["start_frame_id"] } to {b["end_frame_id"] }, scenes { b["scene_ids"][0] } to { b["scene_ids"][-1] }, time { b["start_ms"]} to { b["end_ms"] } =======\n')
    if len(b["text"]) > 0: 
        print(f'\nChapter Text: { b["text"] }')
    else:
        print(f'\nChapter Text (conversation topic): None')

    video['frames'].display_frames(start=b['start_frame_id'], end=b['end_frame_id']+1)

    # ALTERNATIVE: view the composite images that will be used in prompts
    #for image_file in b['composite_images']:
    #    display(DisplayImage(filename=image_file['file'], height=100))
    #if counter == STOP:
    #    break

# Generate chapter level contextual information 

The last step is to send both the visually and audio-aligned data to Claude 3 Sonnet to generate contextual information for each topic. This approach that takes advantage of the multimodal capabilities of the Claude 3 family of models. From our testing, these models have demonstrated the ability to capture minute details from large images and follow image sequences when provided with appropriate instructions.

To prepare the input for Claude3 Sonnet, we first assemble video frames associated with each topic and create a composite image grid. Through our experimentation, we have found that the optimum image grid ratio is 7 rows by 4 columns, which will assemble a 1568 x 1540 pixel image that fits under Claude's 5 MB image file size limit while still preserving enough detail in each individual frame tile. Furthermore, you can also assemble multiple images if needed.

Subsequently, the composite images, the transcription, the IAB Content taxonomy definitions, and GARM taxonomy definitions are fed into the prompt to generate descriptions, sentiment, IAB taxonomy, GARM taxonomy, and other relevant information in a single query to the Claude3 Haiku model. Not only that, but we can adapt this approach to any taxonomy or custom labeling use cases without the need to train a model each time. This is where the true power of this approach lies. The final output can be presented to a human reviewer for final confirmation if needed. Here is an example of a composite image grid and the corresponding contextual output for a specific topic.

![Contextualized chapters](./static/images/02-chapter-contextualization.png)

## Download the IAB Content Taxonomy definition

In [ ]:
iab_file = 'iab_content_taxonomy_v3.json'
url = f"https://dx2y1cac29mt3.cloudfront.net/iab/{iab_file}"

!curl {url} -o {iab_file}

In [ ]:
def load_iab_taxonomies(file):
    with open(file) as f:
        iab_taxonomies = json.load(f)
    return iab_taxonomies


## Generate contextual metadata for each chapter segment

In [ ]:
import time

total_usage = {
    'input_tokens': 0,
    'output_tokens': 0,
}

iab_definitions = load_iab_taxonomies(iab_file)

for chapter in video['chapters'].chapters:

    composite_images = chapter['composite_images']
    num_images = len(composite_images)

    chapter_id = chapter['id']
    text = chapter['text'] 

    contextual_response = brh.get_contextual_information(composite_images, chapter['text'], iab_definitions)
    time.sleep(5)
    usage = contextual_response['usage']
    contextual = contextual_response['content'][0]['json']

    # save the contextual to the chapter
    chapter['contextual'] = {
        'usage': usage,
        **contextual
    }
    
    total_usage['input_tokens'] += usage['input_tokens']
    total_usage['output_tokens'] += usage['output_tokens']

    print(f"==== Chapter #{chapter['id']:02d}: Contextual information ======")
    video['frames'].display_frames(start=chapter['start_frame_id'], end=chapter['end_frame_id']+1)
    for key in ['description', 'sentiment', 'iab_taxonomy', 'garm_taxonomy']:
        print(f"{key.capitalize()}: {colored(contextual[key]['text'], 'green')} ({contextual[key]['score']}%)")

    for key in ['brands_and_logos', 'relevant_tags']:
        items = ', '.join([item['text'] for item in contextual[key]])
        if len(items) == 0:
            items = 'None'
        print(f"{key.capitalize()}: {colored(items, 'green')}")
    print(f"================================================\n\n")

output_file = os.path.join(video["output_dir"], 'scenes_in_chapters.json')
util.save_to_file(output_file, video['chapters'].chapters)

contextual_cost = brh.display_contextual_cost(total_usage)

## Ad breaks

At this point, we have generated video segments that have clear visual breaks between scenes, and we have grouped the scenes into chapters that have clear breaks between topics from the speech in the audio. The breaks between chapters are all candidate ad placement opportunities.  We can use the IAB taxonomy of the chapter segments adjacent to the breaks to make better decisions about what ad to place in that break.  

As you view the chapter segments, imagine yourself as a company who wants to advertise your brand, are there breaks you prefer over others in terms of brand safety?

Now imagine you are a viewer.  What products would be interesting if you chose this title?

In practice, ad breaks would be ranked by a value function that takes into account the needs of the consumer, the publisher, and the advertisers.


## Visualize the ad breaks 

In this section, we'll insert a test Ad into one of the breaks to visualize the ad experience.  You can change the value of BREAK_CHAPTER_ID to try different chapter breaks.

In [ ]:
import moviepy
from moviepy.editor import VideoFileClip, concatenate_videoclips

BREAK_CHAPTER_ID = 2

ad_demo_file= f"ad_break_{ BREAK_CHAPTER_ID }_demo.mp4"
adbreak_start = video['chapters'].chapters[BREAK_CHAPTER_ID]['start_ms']/1000

clip1 = VideoFileClip(video["path"], target_resolution=(360, 640)).subclip(adbreak_start-10, adbreak_start)
clip2 = VideoFileClip("static/images/CountdownClock_0.mp4", target_resolution=(360, 640))
clip3 = VideoFileClip(video["path"], target_resolution=(360, 640)).subclip(adbreak_start, adbreak_start+10)
final_clip = concatenate_videoclips([clip1,clip2,clip3], method="compose")
final_clip.write_videofile(ad_demo_file)

In [ ]:
Video(url=ad_demo_file, width=640, height=360)

